In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np
from torch.utils.data import Dataset, DataLoader
import glob
import matplotlib.pyplot as plt
import os

# 3D CNN Model
class CNN3D(nn.Module):
    def __init__(self):
        super(CNN3D, self).__init__()
        
        self.conv0 = nn.Conv3d(in_channels=37, out_channels=64, kernel_size=1, stride=1, padding=0) # play around with output channels
        self.conv1 = nn.Conv3d(in_channels=64, out_channels=64, kernel_size=3, stride=1, padding=1)
        self.pool = nn.MaxPool3d(kernel_size=2, stride=2, padding=0)
        self.conv2 = nn.Conv3d(in_channels=64, out_channels=128, kernel_size=3, stride=1, padding=1)
        self.pool2 = nn.MaxPool3d(kernel_size=2, stride=2, padding=0)

        #self.dropout_conv = nn.Dropout3d(p=0.05)
        
        # After two pooling layers, spatial dimensions reduce from 40x40x40 -> 5x5x5
        self.fc1 = nn.Linear(128 * 3 * 3 * 3, 256)  # Try increasing over 256
        self.fc2 = nn.Linear(256, 128)
        self.fc3 = nn.Linear(128, 2)  # Assuming 1 output for docking status/position

        #self.dropout_fc = nn.Dropout(p=0.15)
        
    def forward(self, x):
        # Forward pass through Conv layers
        x = self.pool(torch.relu(self.conv0(x)))  # Conv0 -> ReLU -> Pooling
        #x = self.dropout_conv(x)
        x = self.pool(torch.relu(self.conv1(x)))  # Conv1 -> ReLU -> Pooling
        x = self.pool2(torch.relu(self.conv2(x)))  # Conv2 -> ReLU -> Pooling

        # Flatten the input for fully connected layers
        x = x.view(-1, 128 * 3 * 3 * 3)
        
        # Forward pass through fully connected layers
        x = torch.relu(self.fc1(x)) #use tanh activation
        #x = self.dropout_fc(x)
        x = torch.relu(self.fc2(x))
        x = torch.nn.functional.softmax(self.fc3(x), dim=1)  # Final layer (output layer)
        #x = torch.clamp(x, min=1e-7, max=1 - 1e-7)  # Clamp outputs to avoid extreme values
        
        return x

# Set device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

models = []

for i in range (1, 50):
    # Initialize model
    model = CNN3D().to(device)

    # Load saved model weights
    model_path = f"3DCholesterolModels-st_exp1/model_bin_{i}.pth"  # Update with correct path if needed
    model.load_state_dict(torch.load(model_path, map_location=device))

    # Set the model to evaluation mode
    model.eval()

    models.append(model)


/tmp/ipykernel_3811908/950283952.py:60: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load(model_path, map_location=device))


In [2]:
import numpy as np

def evaluate_file(model, file_path):
    # Load numpy array and convert it to a PyTorch tensor
    grid = np.load(file_path)
    grid_tensor = torch.tensor(grid, dtype=torch.float32).permute(3, 0, 1, 2).unsqueeze(0)  # Add batch dimension
    grid_tensor = grid_tensor.to(device)
    
    # Perform inference
    with torch.no_grad():
        output = model(grid_tensor)
    
    # Get predicted class (assuming softmax output)
    predicted_class = torch.argmax(output, dim=1).item() # try different thresholds and save predictions as well into excel sheet
    confidence = torch.max(output).item()
    positive_class_prob = output[0, 1].item()

    return predicted_class, confidence, positive_class_prob

# # Example usage:
# file_path = "path_to_your_file.npy"  # Update with the actual file path
# pred_class, conf = evaluate_file(model, file_path)
# print(f"Predicted Class: {pred_class}, Confidence: {conf:.4f}")


In [3]:
import glob

def evaluate_directory(model, directory):
    files = glob.glob(f"{directory}/*.npy")  # Adjust path as needed
    results = {}

    false_negative_files = []
    for file in files:
        pred_class, conf, probability = evaluate_file(model, file)
        results[file] = {"Predicted Class": pred_class, "Confidence": conf, "Probability": probability}
        if pred_class == 0:
            false_negative_files.append(file)

    return results, false_negative_files

directory_path = "../../../Data/SplitData/Cholesterol/cholesterol-grid-st_exp1/Spies"  # Update with your validation directory
print(f"{'ModelID':<10}{'#ofPos':<10}{'#ofNeg':<10}{'UnconfPos':<12}{'UnconfNeg':<12}{'PositiveRatio':<12}")

file_labels = {}
file_raw_softmax = {}

model_id = 1
total_false_negative_files = []
for model in models:
    # Example usage:
    evaluation_results, false_negative_files = evaluate_directory(model, directory_path)
    total_false_negative_files.extend(false_negative_files)

    # Print results
    positives_count = 0
    unconfident_positive_count = 0
    false_negative_count = 0
    unconfident_negative_count = 0
    for file, result in evaluation_results.items():
        if file not in file_labels:
            file_labels[file] = 0
        file_labels[file] += result['Probability']
        
        #print(f"File: {file}, Prediction: {result['Predicted Class']}, Confidence: {result['Confidence']:.4f}")
        if result['Predicted Class'] == 1:
            positives_count += 1
            if result['Confidence'] < 0.85:
                unconfident_positive_count += 1
        if result['Predicted Class'] == 0:
            false_negative_count += 1
            if result['Confidence'] < 0.85:
                unconfident_negative_count += 1
    print(f"{model_id:<10}{positives_count:<10}{false_negative_count:<10}{unconfident_positive_count:<12}{unconfident_negative_count:<12}{(positives_count / 770):<12}")
    model_id += 1

overall_spy_capture = 0
print(f"\n{'FileName':<70}{'Labels':<4}")
for file, total_score in file_labels.items():
    average_score = total_score/ 50
    print(f"{file:<70} {average_score:<4}")
    overall_spy_capture += average_score

print("Overall Spy Capture Rate:", (overall_spy_capture / 770))




ModelID   #ofPos    #ofNeg    UnconfPos   UnconfNeg   PositiveRatio
1         712       58        42          20          0.9246753246753247
2         711       59        41          26          0.9233766233766234
3         701       69        47          26          0.9103896103896104
4         720       50        53          15          0.935064935064935
5         715       55        46          20          0.9285714285714286
6         710       60        36          22          0.922077922077922
7         706       64        42          27          0.9168831168831169
8         708       62        41          19          0.9194805194805195
9         703       67        40          29          0.912987012987013
10        711       59        49          19          0.9233766233766234
11        717       53        36          19          0.9311688311688312
12        709       61        49          22          0.9207792207792208
13        715       55        38          20          0.928

In [5]:
# test on propofol positive grids
directory_path = "../../../Data/SplitData/Cholesterol/cholesterol-grid-st_exp1/Test/CombinedUnlabeled"  # Update with your validation directory
print(f"{'ModelID':<10}{'#ofPos':<10}{'#ofNeg':<10}{'UnconfPos':<12}{'UnconfNeg':<12}{'PositiveRatio':<12}")

file_labels = {}
file_raw_softmax = {}

model_id = 1
total_false_negative_files = []
for model in models:
    # Example usage:
    evaluation_results, false_negative_files = evaluate_directory(model, directory_path)
    total_false_negative_files.extend(false_negative_files)

    # Print results
    positives_count = 0
    unconfident_positive_count = 0
    false_negative_count = 0
    unconfident_negative_count = 0
    for file, result in evaluation_results.items():
        if file not in file_labels:
            file_labels[file] = 0
        file_labels[file] += result['Probability']
        
        #print(f"File: {file}, Prediction: {result['Predicted Class']}, Confidence: {result['Confidence']:.4f}")
        if result['Predicted Class'] == 1:
            positives_count += 1
            if result['Confidence'] < 0.85:
                unconfident_positive_count += 1
        if result['Predicted Class'] == 0:
            false_negative_count += 1
            if result['Confidence'] < 0.85:
                unconfident_negative_count += 1
    print(f"{model_id:<10}{positives_count:<10}{false_negative_count:<10}{unconfident_positive_count:<12}{unconfident_negative_count:<12}{(positives_count / 4610):<12}")
    model_id += 1

accurate_files = []
positive_capture_rate = 0

print(f"\n{'FileName':<70}{'Labels':<4}")
for file, total_score in file_labels.items():
    print(f"{file:<70} {(total_score/ 50):<4}")
    positive_capture_rate += total_score / 50
    
    if (total_score / 50) > 0.8:
        accurate_files.append(file)
print(accurate_files)

print("Overall Positive Capture Rate is:", (positive_capture_rate / 4610))


ModelID   #ofPos    #ofNeg    UnconfPos   UnconfNeg   PositiveRatio
1         1811      2799      342         325         0.3928416485900217
2         1791      2819      364         370         0.38850325379609546
3         1846      2764      318         288         0.40043383947939265
4         1707      2903      362         381         0.3702819956616052
5         1906      2704      348         262         0.4134490238611714
6         1679      2931      336         326         0.36420824295010845
7         1830      2780      326         274         0.3969631236442516
8         1660      2950      299         296         0.3600867678958785
9         1754      2856      281         311         0.38047722342733187
10        1727      2883      319         313         0.37462039045553147
11        1755      2855      328         329         0.3806941431670282
12        1811      2799      354         306         0.3928416485900217
13        1720      2890      301         326      

In [6]:
# test on propofol positive grids
directory_path = "../../../Data/SplitData/Cholesterol/cholesterol-grid-st_exp1/Test/Positive"  # Update with your validation directory
print(f"{'ModelID':<10}{'#ofPos':<10}{'#ofNeg':<10}{'UnconfPos':<12}{'UnconfNeg':<12}{'PositiveRatio':<12}")

file_labels = {}
file_raw_softmax = {}

model_id = 1
total_false_negative_files = []
for model in models:
    # Example usage:
    evaluation_results, false_negative_files = evaluate_directory(model, directory_path)
    total_false_negative_files.extend(false_negative_files)

    # Print results
    positives_count = 0
    unconfident_positive_count = 0
    false_negative_count = 0
    unconfident_negative_count = 0
    for file, result in evaluation_results.items():
        if file not in file_labels:
            file_labels[file] = 0
        file_labels[file] += result['Probability']
        
        #print(f"File: {file}, Prediction: {result['Predicted Class']}, Confidence: {result['Confidence']:.4f}")
        if result['Predicted Class'] == 1:
            positives_count += 1
            if result['Confidence'] < 0.85:
                unconfident_positive_count += 1
        if result['Predicted Class'] == 0:
            false_negative_count += 1
            if result['Confidence'] < 0.85:
                unconfident_negative_count += 1
    print(f"{model_id:<10}{positives_count:<10}{false_negative_count:<10}{unconfident_positive_count:<12}{unconfident_negative_count:<12}{(positives_count / 770):<12}")
    model_id += 1

accurate_files = []
positive_capture_rate = 0

print(f"\n{'FileName':<70}{'Labels':<4}")
for file, total_score in file_labels.items():
    print(f"{file:<70} {(total_score/ 50):<4}")
    positive_capture_rate += total_score / 50
    
    if (total_score / 50) > 0.8:
        accurate_files.append(file)
print(accurate_files)

print("Overall Positive Capture Rate is:", (positive_capture_rate / 770))


ModelID   #ofPos    #ofNeg    UnconfPos   UnconfNeg   PositiveRatio
1         723       47        34          21          0.938961038961039
2         718       52        37          27          0.9324675324675324
3         719       51        34          18          0.9337662337662338
4         727       43        49          20          0.9441558441558442
5         720       50        34          18          0.935064935064935
6         727       43        34          20          0.9441558441558442
7         714       56        32          22          0.9272727272727272
8         716       54        30          20          0.9298701298701298
9         718       52        32          18          0.9324675324675324
10        718       52        34          19          0.9324675324675324
11        727       43        33          16          0.9441558441558442
12        718       52        32          27          0.9324675324675324
13        725       45        35          18          0.94